In [1]:
import os, pickle, sys, torch, numpy as np

# paths  
# table_path = '/root/gurusmart/MorganFP_prediction/reproduce_previous_works/all_2d1d_datasets/entropy_radius_exps_on_HSQC'
# table_path = "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/train_on_all_data_possible_with_jittering"
table_path = "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/all_HSQC_jittering_search"

print_keys = [
    # "test/mean_rank_1",
    # "test/mean_rank_5",
    # "test/mean_rank_10",
    # "test/mean_mean_rank",
    "test/mean_cos",
    # "test/mean_f1", 
]


The first part is for different kinds of NMR in the all info set (kinda depreated now)


The second part is for training with all possible data of a specific type.

In [3]:
from collections import defaultdict
exp_name_to_key_to_results = defaultdict(lambda: defaultdict(list))
# for input_type in ['1d', "all_info", "HSQC_and_C", "HSQC_and_H", "only_C", "only_H", "only_HSQC"]:

for input_type in ['all_info']:    
    # table_path = f'/root/gurusmart/MorganFP_prediction/reproduce_previous_works/puuting_h_in_the_middle/entropy_radius_exps_{input_type}'
    # table_path = f'/root/gurusmart/MorganFP_prediction/reproduce_previous_works/rank_on_entire_set/entropy_radius_exps_{input_type}'


    all_exps = os.listdir(table_path)
    # load pickles 
    for exp in all_exps:
        if 'trial' in exp:
            trial_spelling = "trial"
        elif 'trail' in exp:
            trial_spelling = "trail"
           
        try:     
            # print(exp)
            with open(f'{table_path}/{exp}/test_result.pkl', 'rb') as f:
                result = pickle.load(f)[0]
                
                for key in print_keys:
                    if key not in result:
                        continue
                    exp_name_to_key_to_results["_".join(exp.split(trial_spelling)[:-1])][key].append(result[key])
                    # exp_name_to_key_to_results[exp][key].append(result[key])
                    # print(key, result[key])
        except Exception as e:
            print(f'{table_path}/{exp}/test_result.pkl')            
    
exp_name_to_key_to_results


defaultdict(<function __main__.<lambda>()>,
            {'only_hsqc_jittering_0.5-': defaultdict(list,
                         {'test/mean_cos': [0.893612265586853,
                           0.8954458236694336,
                           0.8943120241165161]}),
             'only_hsqc_jittering_0.25-': defaultdict(list,
                         {'test/mean_cos': [0.9002151489257812,
                           0.8988407850265503,
                           0.8999898433685303]}),
             'only_hsqc_jittering_2-': defaultdict(list,
                         {'test/mean_cos': [0.8553824424743652,
                           0.8593199849128723,
                           0.8590157628059387]}),
             'only_hsqc_jittering_0-': defaultdict(list,
                         {'test/mean_cos': [0.9046812653541565,
                           0.90409916639328,
                           0.9011514186859131]}),
             'only_hsqc_jittering_1-': defaultdict(list,
                         

In [3]:
# # compute the std  of rank_1 and rank_5 for each nmr
# for exp_name, key_to_results in exp_name_to_key_to_results.items():
#     for key, results in key_to_results.items():
#         if key in ["test/mean_rank_1", "test/mean_rank_5"]:
#             print(exp_name, key, np.mean(results), np.std(results))
#         # print(exp_name, key, np.mean(results), np.std(results))
#     print()

In [4]:

avg_results = {}
for exp_name, key_to_results in exp_name_to_key_to_results.items():
    avg_results[exp_name] = {key:[ np.mean(val), np.std(val) ]for key, val in key_to_results.items()}

In [5]:
# sort based on dict key 
def get_sort_key(item):
    if ((item[0].split("R0_to_R")[-1]).split("_")[0]).isdecimal():
        return int((item[0].split("R0_to_R")[-1]).split("_")[0])
    
    else:
        return float('inf')



avg_results = dict(sorted(avg_results.items(),key=get_sort_key ))


In [6]:
avg_results


{'only_hsqc_jittering_0.5-': {'test/mean_cos': [np.float64(0.894456704457601),
   np.float64(0.0007555056193796932)]},
 'only_hsqc_jittering_0.25-': {'test/mean_cos': [np.float64(0.8996819257736206),
   np.float64(0.0006018465838946809)]},
 'only_hsqc_jittering_2-': {'test/mean_cos': [np.float64(0.8579060633977255),
   np.float64(0.0017887863103610742)]},
 'only_hsqc_jittering_0-': {'test/mean_cos': [np.float64(0.9033106168111166),
   np.float64(0.001545167133996608)]},
 'only_hsqc_jittering_1-': {'test/mean_cos': [np.float64(0.8851240277290344),
   np.float64(0.0018626676297420661)]}}

In [7]:
name_convert = {
    "all_info": "All 3 NMRs",
    "1d": "13C and 1H",
    "HSQC_and_C": "HSQC and 13C",
    "HSQC_and_H": "HSQC and 1H",
    "only_c": "Only 13C",
    "only_H": "Only 1H",
    "only_hsqc": "Only HSQC",
}

# for each combination, find which fp is best
for combination in ["all_info", '1d', "HSQC_and_C", "HSQC_and_H", "only_c", "only_H", "only_hsqc"]:
    print(name_convert[combination], end="")
    for mfp_type in ["R0_to_R1_", "R0_to_R2", "R0_to_R3", "R0_to_R4", "R0_to_R5", "Hyun_FP"]:
        for key, metric_dict in avg_results.items():
            # print(key)
            if combination not in key or mfp_type not in key:
                continue
            for metric, (mean,std) in metric_dict.items():
                if metric in ["test/mean_rank_1", ]:
                    print(f' & {mean*100:.1f}\%\\textpm{std*100:.1f}\%', end="")
    print(" \\\\")
    print("\hline")
    

All 3 NMRs \\
\hline
13C and 1H \\
\hline
HSQC and 13C \\
\hline
HSQC and 1H \\
\hline
Only 13C \\
\hline
Only 1H \\
\hline
Only HSQC \\
\hline


In [8]:
for input_type in ["all_info", '1d', "HSQC_and_C", "HSQC_and_H", "only_c", "only_H", "only_hsqc"]:
    print("\n\n")
    # print in latex format
    print ("\\begin{table}[h]")
    print("\centering")
    print("\\resizebox{\\textwidth}{!}{")
    print('\\begin{tabular}{|c|c|c|c|c|c|}')
    print("\hline")
    print("Model Input & Rank-1$\\uparrow$ & Rank-5$\\uparrow$ & Mean Rank$\\downarrow$ & Cosine Sim$\\uparrow$ & F1-score$\\uparrow$\\\\")
    print("\hline")

    for exp_name, key_to_results in avg_results.items():
        if input_type not in exp_name:
            continue
        
        name = exp_name
        if name[:2]=="FP":
            name = "R"+name.split(" ")[-1][-2]+" FP"
        MF_name = " ".join(name.split("_")[:3]) + " MF"
        if MF_name.startswith("Hyun"):
            MF_name = "DeepSAT FP"
        print(MF_name, end="")
        for key, (mean, std) in key_to_results.items():
            if key in ["test/mean_rank_1", "test/mean_rank_5"]:
                print(f' & {mean*100:.2f}\%\\textpm{std*100:.2f}\%', end="")
            elif key in ["test/mean_mean_rank"]:
                print(f' & {mean+1:.2f}\\textpm{std:.2f}', end="")
            else:
                print(f' & {mean:.4f}\\textpm{std:.4f}', end="")
        print(" \\\\")
        print("\hline")
        
    print('\end{tabular}')
    print('}')
    print("\caption{Performance of various MFs when "+ name_convert[input_type] +" is available}")
    print('\label{table_name}')
    print('\end{table}')




\begin{table}[h]
\centering
\resizebox{\textwidth}{!}{
\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model Input & Rank-1$\uparrow$ & Rank-5$\uparrow$ & Mean Rank$\downarrow$ & Cosine Sim$\uparrow$ & F1-score$\uparrow$\\
\hline
\end{tabular}
}
\caption{Performance of various MFs when All 3 NMRs is available}
\label{table_name}
\end{table}



\begin{table}[h]
\centering
\resizebox{\textwidth}{!}{
\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model Input & Rank-1$\uparrow$ & Rank-5$\uparrow$ & Mean Rank$\downarrow$ & Cosine Sim$\uparrow$ & F1-score$\uparrow$\\
\hline
\end{tabular}
}
\caption{Performance of various MFs when 13C and 1H is available}
\label{table_name}
\end{table}



\begin{table}[h]
\centering
\resizebox{\textwidth}{!}{
\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model Input & Rank-1$\uparrow$ & Rank-5$\uparrow$ & Mean Rank$\downarrow$ & Cosine Sim$\uparrow$ & F1-score$\uparrow$\\
\hline
\end{tabular}
}
\caption{Performance of various MFs when HSQC and 13C is available}
\label{table_name}
\e

In [9]:
exp_name

NameError: name 'exp_name' is not defined

Look at All_possible_data


In [1]:
print_keys = [
    "test/mean_rank_1",
    "test/mean_rank_5",
    "test/mean_rank_10",
    # "test/mean_mean_rank",
    "test/mean_cos",
    "test/mean_f1", 
]

import os, pickle, sys, torch, numpy as np

from collections import defaultdict
exp_name_to_key_to_results = defaultdict(lambda: defaultdict(list))
# for input_type in ['1d', "all_info", "HSQC_and_C", "HSQC_and_H", "only_C", "only_H", "only_HSQC"]:
for input_type in ['']:    
    # table_path = f'/root/gurusmart/MorganFP_prediction/reproduce_previous_works/stop_on_cosine/all_data_possible'
    # table_path = "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/rank_on_entire_set/all_data_possible"
    table_path = "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/train_on_all_data_possible_with_jittering"



    all_exps = os.listdir(table_path)
    # load pickles 
    for exp in all_exps:
        if 'trial' in exp:
            trial_spelling = "trial"
        elif 'trail' in exp:
            trial_spelling = "trail"
           
        try:     
            # print(exp)
            with open(f'{table_path}/{exp}/test_result.pkl', 'rb') as f:
                result = pickle.load(f)[0]
                
                for key in print_keys:
                    if key not in result:
                        continue
                    exp_name_to_key_to_results["_".join(exp.split(trial_spelling)[:-1])][key].append(result[key])
                    # exp_name_to_key_to_results[exp][key].append(result[key])
                    # print(key, result[key])
        except Exception as e:
            print(f'{table_path}/{exp}/test_result.pkl')            
    
# exp_name_to_key_to_results

/root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/train_on_all_data_possible_with_jittering/retest/test_result.pkl


In [2]:
avg_results = {}
std_results = {}
for exp_name, key_to_results in exp_name_to_key_to_results.items():
    avg_results[exp_name] = {key: (np.mean(val), np.std(val)) for key, val in key_to_results.items()}
    

In [3]:
avg_results.keys()

dict_keys(['only_hsqc_', 'only_1d_', 'no_mw_only_c_', 'HSQC_and_C_', 'only_c_', 'no_mw_all_info_', 'no_mw_normal_HSQC_and_H_', 'HSQC_and_H_', 'only_normal_hsqc_', 'normal_HSQC_and_H_', 'no_mw_all_info_normal_HSQC_', 'all_info_normal_HSQC_', 'only_h_', 'no_mw_HSQC_and_C_', 'no_mw_only_h_', 'all_info_', 'normal_HSQC_and_C_', 'no_mw_only_hsqc_', 'no_mw_only_1d_', 'no_mw_HSQC_and_H_', 'no_mw_normal_HSQC_and_C_', 'no_mw_only_normal_hsqc_'])

In [4]:



# for k in ['all_info_', 'only_1d_',  'HSQC_and_C_', 'HSQC_and_H_', 'only_c_',  'only_h_',  'only_hsqc_' ]:
#     print(k)
#     for metric in print_keys:
#         print(metric)
#         mean,std = (avg_results[k][metric])
#         print(f"{mean*100:.2f}\%\\textpm{std*100:.2f}\% ")
#     print()
#     # mean,std = (avg_results[k]['test/mean_rank_1'])
#     # print(f"{mean*100:.2f}\%\\textpm{std*100:.2f}\% ")

In [5]:



for k in avg_results.keys():
    renamed_k = k
    
    if ("hsqc" in renamed_k or "HSQC" in renamed_k) and "normal" not in renamed_k:
            renamed_k = renamed_k.replace("hsqc","HSQC").replace("HSQC", "edited HSQC")
    renamed_k = renamed_k.replace("normal", 'standard')    
    print(renamed_k)
    for metric in print_keys:
        
        print(metric)
        mean,std = (avg_results[k][metric])
        print(f"{mean*100:.2f}\%\\textpm{std*100:.2f}\% ")
    print()
    # mean,std = (avg_results[k]['test/mean_rank_1'])
    # print(f"{mean*100:.2f}\%\\textpm{std*100:.2f}\% ")

only_edited HSQC_
test/mean_rank_1
45.84\%\textpm0.97\% 
test/mean_rank_5
79.08\%\textpm0.21\% 
test/mean_rank_10
85.84\%\textpm0.33\% 
test/mean_cos
87.14\%\textpm0.15\% 
test/mean_f1
88.66\%\textpm0.14\% 

only_1d_
test/mean_rank_1
37.20\%\textpm0.32\% 
test/mean_rank_5
71.42\%\textpm0.44\% 
test/mean_rank_10
79.83\%\textpm0.20\% 
test/mean_cos
83.79\%\textpm0.10\% 
test/mean_f1
85.62\%\textpm0.07\% 

no_mw_only_c_
test/mean_rank_1
34.08\%\textpm0.00\% 
test/mean_rank_5
67.12\%\textpm0.00\% 
test/mean_rank_10
76.71\%\textpm0.00\% 
test/mean_cos
82.30\%\textpm0.00\% 
test/mean_f1
84.85\%\textpm0.00\% 

edited HSQC_and_C_
test/mean_rank_1
44.13\%\textpm0.27\% 
test/mean_rank_5
76.14\%\textpm0.41\% 
test/mean_rank_10
84.14\%\textpm0.02\% 
test/mean_cos
84.85\%\textpm0.18\% 
test/mean_f1
86.29\%\textpm0.17\% 

only_c_
test/mean_rank_1
33.02\%\textpm0.50\% 
test/mean_rank_5
66.87\%\textpm0.26\% 
test/mean_rank_10
75.67\%\textpm0.10\% 
test/mean_cos
81.96\%\textpm0.35\% 
test/mean_f1
84.18

In [6]:
avg_results.keys()

dict_keys(['only_hsqc_', 'only_1d_', 'no_mw_only_c_', 'HSQC_and_C_', 'only_c_', 'no_mw_all_info_', 'no_mw_normal_HSQC_and_H_', 'HSQC_and_H_', 'only_normal_hsqc_', 'normal_HSQC_and_H_', 'no_mw_all_info_normal_HSQC_', 'all_info_normal_HSQC_', 'only_h_', 'no_mw_HSQC_and_C_', 'no_mw_only_h_', 'all_info_', 'normal_HSQC_and_C_', 'no_mw_only_hsqc_', 'no_mw_only_1d_', 'no_mw_HSQC_and_H_', 'no_mw_normal_HSQC_and_C_', 'no_mw_only_normal_hsqc_'])

get the validation cosine score

In [7]:
import numpy as np

import os
import re
from collections import defaultdict
import numpy as np

base_dir = "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/all_HSQC_jittering_search"
pattern = re.compile(r'only_hsqc_jittering_([0-9.]+)-trial-\d+')

# Dictionary to collect scores for each jittering value
jittering_scores = defaultdict(list)

for subfolder in os.listdir(base_dir):
    match = pattern.match(subfolder)
    if match:
        jittering_val = float(match.group(1))
        log_path = os.path.join(base_dir, subfolder, "logs.txt")
        if os.path.isfile(log_path):
            with open(log_path, 'r') as f:
                for line in f:
                    if "best score:" in line:
                        try:
                            score = float(line.strip().split("best score:")[-1])
                            jittering_scores[jittering_val].append(score)
                        except ValueError:
                            print(f"Could not parse score in {log_path}")
                        break  # Assuming only one "best score:" line per file

# Print results
for jittering_val in sorted(jittering_scores):
    scores = np.array(jittering_scores[jittering_val])
    mean = np.mean(scores)
    std = np.std(scores)
    print(f"Jittering {jittering_val}: Mean = {mean:.4f}, Std = {std:.4f}")


Jittering 0.0: Mean = 0.9025, Std = 0.0009
Jittering 0.25: Mean = 0.8981, Std = 0.0023
Jittering 0.5: Mean = 0.8954, Std = 0.0008
Jittering 1.0: Mean = 0.8856, Std = 0.0003
Jittering 2.0: Mean = 0.8616, Std = 0.0013
